# Предварительный разбор

В ходе теста (без учета короткой тренировки в начале) было предъявлено 6 заданий, 
      в которых надо было нажимать на клавиши в ускоряющемся темпе.
      
[Онлайн-отчет](http://stireac.com/uskor/result/sherdim%40gmail.com/10.132.32.111__6158019760000190311)



In [ ]:
from __future__ import print_function, division, unicode_literals
%pylab inline
from ipywidgets import interact

In [ ]:
import pandas as pd

In [ ]:
u='http://stireac.com/uskor/result.tsv/sherdim%40gmail.com/10.132.32.111__6158019760000190311'
M = pd.read_table(u)
M.info()

In [ ]:
M.plot('t','v');

Попадаются какие-то огромные значения... это служебные маркеры.

In [ ]:
M.v.value_counts()

Чаще всего встречаются маркеры 70 и 74. А в тесте надо было нажимать на клавиши F и J. Совпадение?

In [ ]:
chr(70), chr(74)

Есть команда и для обратного перевода. 

In [ ]:
ord('F')

На самом деле код маркеров нажатий не обязательно должен совпадать с кодом символа. В стандартной клавиатуре, где одна кнопка - один символ, - это ествественно. Но, например, кнопки игровых джойстиков не имеют специальных названий и нумеруются произвольно.

Оставим только маркеры, не превышающие 7000.

In [ ]:
M[M.v.abs() < 7000].plot('t','v');

Из рисунка видно, что первая часть теста проходила примерно с 30 по 45 с.

In [ ]:
M[ (25 < M.t) & (M.t < 48)]

В тест добавлены маркеры начала этапов: 1001, 1002, ...

Ориентируясь по ним, мы можем извлечь данные нужного нам этапа.

In [ ]:
coab, coad = 1001, 1002
tab = M.t[M.v==coab].iloc[0]
tad = M.t[M.v==coad].iloc[0]
R = M[(M.t>=tab)&(M.t<tad)].copy()
# оставим только клавиши - код < 100
R = R[R.v.abs()<100]
R.plot('t','v');

Отжатия клавиши нужны только для оценки длительности удержания и не влияют на ритм нажатий - ту характеристику, которая нас интересует. Или влияют?

In [ ]:
# Убедимся, что количество нажатий и отжатий совпадает
assert(R.t[R.v > 0].shape == R.t[R.v < 0].shape)

R.loc[R.v > 0, 'toff'] = R.t.values[R.v < 0]

R

Теперь можно удалить строки с отжатиями, и последнее нажатие на клавишу Enter нас в данном случае не интересует.

In [ ]:
R.dropna(inplace=True)
R.drop(R.index[-1], inplace=True)

R.v.value_counts()

Итак, у нас остались нажатия на одну кнопку. Посчитаем интервал между нажатиями.

In [ ]:
R['iri'] = R.t.diff()
plot(R.iri, '.');

## Скользящее среднее

Самый простой способ сглаживания сигнала - это замена исходных значений на среднее нескольких соседних значений в небольшом окне.

Библиотека `pandas` имеет встроенные средства для расчета значений в "скользящем окне", или "катящемся" (rolling) вдоль ряда блоке.

In [ ]:
@interact(win=(3,15))
def _plot(win=5):
    trend = R.iri.rolling(win).mean()  #, min_periods=1, center=True
    plot(R.t, R.iri, '.', label='Интервалы');
    plot(R.t, trend, 'r-', label='Тренд');
    legend();

1. Добавьте параметры в метод `rolling` и изучите как это влияет на форму тренда.
2. Попробуйте поменять `mean` (среднюю) на `median` (медиану) - более надежную меру центральной тенденции. Как это повлияет на стабильность тренда? Почему?